In [ ]:
import os
import glob
import tifffile
import numpy as np
import pandas as pd

# Préparer les données

### **Lecture de la série temporelle d'images et Normalisation par bande sur la série temporelle entre 0 et 1** 

#### **Lecture des images et création des séries temporelles de bandes**

In [ ]:
# Récupérer la liste des images
lst_img = glob.glob('Data/Images/*.tif')
lst_img.sort()
lst_img

In [ ]:
# Lecture de la bande du rouge (B1) pour toute la série temporelle
red_ts = []
for img in lst_img:
    red_ts.append(tifffile.imread(img)[:,:,0])
red_ts = np.dstack(red_ts)
red_ts.shape

In [ ]:
# Lecture de la bande du vert (B2) pour toute la série temporelle
green_ts = []
for img in lst_img:
    green_ts.append(tifffile.imread(img)[:,:,1])
green_ts = np.dstack(green_ts)
green_ts.shape

In [ ]:
# Lecture de la bande du bleu (B3) pour toute la série temporelle
blue_ts = []
for img in lst_img:
    blue_ts.append(tifffile.imread(img)[:,:,2])
blue_ts = np.dstack(blue_ts)
blue_ts.shape

In [ ]:
# Lecture de la bande du proche infrarouge (B4) pour toute la série temporelle
nir_ts = []
for img in lst_img:
    nir_ts.append(tifffile.imread(img)[:,:,3])
nir_ts = np.dstack(nir_ts)
nir_ts.shape

#### **Normalisation en utilisant le min et le max des bandes sur les séries temporelles**

In [ ]:
# Normalisation des séries temporelles par bande: Rouge
red_ts_norm = ( red_ts - red_ts.min() ) / ( red_ts.max() - red_ts.min() ).astype(np.float32)
del red_ts
print(red_ts_norm.min() , red_ts_norm.max(), red_ts_norm.shape, red_ts_norm.dtype)

In [ ]:
# Normalisation des séries temporelles par bande: Vert
green_ts_norm = ( green_ts - green_ts.min() ) / ( green_ts.max() - green_ts.min() ).astype(np.float32)
del green_ts
print(green_ts_norm.min() , green_ts_norm.max(), green_ts_norm.shape, green_ts_norm.dtype)

In [ ]:
# Normalisation des séries temporelles par bande: Bleu
blue_ts_norm = ( blue_ts - blue_ts.min() ) / ( blue_ts.max() - blue_ts.min() ).astype(np.float32)
del blue_ts
print(blue_ts_norm.min() , blue_ts_norm.max(), blue_ts_norm.shape, blue_ts_norm.dtype)

In [ ]:
# Normalisation des séries temporelles par bande: Proche infrarouge
nir_ts_norm = ( nir_ts - nir_ts.min() ) / ( nir_ts.max() - nir_ts.min() ).astype(np.float32)
del nir_ts
print(nir_ts_norm.min() , nir_ts_norm.max(), nir_ts_norm.shape, nir_ts_norm.dtype)

### **Lecture de la vérité terrain et récupération des positions des pixels du jeu d'entraînement et de test**

#### **Lecture des fichiers de la vérité terrain**

In [ ]:
# Lire le fichier correspondant aux classes d'occupation du sol
gt_class = tifffile.imread ('Data/Verite_terrain/DORDOGNE_VT_CLASS.tif')
gt_class.shape , gt_class.dtype

In [ ]:
# Lire le fichier correspondant aux identifiants
gt_id = tifffile.imread ('Data/Verite_terrain/DORDOGNE_VT_ID.tif')
gt_id.shape, gt_id.dtype

#### **Récupération des positions des pixels d'entraînement et de test**

In [ ]:
# Récupérer les positions des échantillons d'entraînement et test
idx_train_ = np.where ( (gt_id!=0) & (gt_class!=0) )
idx_test = np.where ( (gt_id!=0) & (gt_class==0) )

In [ ]:
# Lecture des identifiants et labels des échantillons d'entraînement
train_id_ = gt_id[idx_train_]
train_y_ = gt_class[idx_train_]
f'échantillons d\'entrainement: {train_y_.shape[0]} pixels, {len(np.unique(train_id_))} objets'

In [ ]:
# Lecture des identifiants et labels des échantillons de test
test_id = gt_id[idx_test]
f'échantillons test: {test_id.shape[0]} pixels, {len(np.unique(test_id))} objets'

### **Création d'un jeu de validation en prenant une partie du jeu d'entraînement**
Pour cela, on s'assure de faire la division en mettant les pixels ayant le même identifiant dans un seul et même lot

In [ ]:
# Dataframe pour créer un jeu de validation
samples = pd.DataFrame({'ID':train_id_,'Class':train_y_})
samples = samples.drop_duplicates(keep='first')
samples.head(5)

In [ ]:
# 30% des échantillons de chaque classe affecté au jeu de validation
train_id = []
valid_id = []
for c in np.unique(samples.Class.values) :
    samples_c = samples.loc[samples.Class==c]
    samples_frac = samples_c.sample(frac=0.7,random_state=1234) 
    train_id.extend( samples_frac.ID.values )
    valid_id.extend( samples_c.drop(samples_frac.index).ID.values )
len(train_id),len(valid_id)

### **Récupération des positions des nouveaux échantillons d'entraînement et de validation**

In [ ]:
# Récupérer les positions des nouveaux échantillons d'entraînement et validation
idx_train = np.where ( np.isin(gt_id,train_id) )
idx_valid = np.where ( np.isin(gt_id,valid_id) )

### **Lire finalement les labels correspondant aux nouveaux échantillons d'entraînement, de validation et test**

In [ ]:
train_y = gt_class[idx_train]
valid_y = gt_class[idx_valid]
test_y = gt_class[idx_test]
train_y.shape, valid_y.shape, test_y.shape

### **Lire finalement les identifiants correspondant aux nouveaux échantillons d'entraînement, de validation et test**

In [ ]:
# utile pour les aggrégations au niveau objet
# train_id_array = gt_id[idx_train] # Pas vraiment nécessaire pour les échantillons d'entraînements
valid_id_array = gt_id[idx_valid]
test_id_array = gt_id[idx_test]
#train_id_array.shape, 
valid_id_array.shape, test_id_array.shape

### **Lire finalement les valeurs des séries temporelles correspondant aux nouveaux échantillons d'entraînement, de validation et test**

In [ ]:
# Un CNN 2D ou spatial requiert un tableau 3D du type (nombre d'échantillons, largeur, hauteur, nombre de features= (nombre de dates x nombre de bandes) )
# Vous pouvez rajouter à la liste des bandes les indices spectraux normalisés que vous aurez calculé

# Training
# 1- précaution pour ne pas considérer les pixels dont on ne peut extraire des patchs car trop près du bord
idx_col = idx_train[0]
idx_row = idx_train[1]
coords = np.column_stack((idx_col,idx_row))
coords = coords[np.where( ( np.isin(coords[:,0],np.arange(2,blue_ts_norm.shape[0]-2,1)) ) & ( np.isin(coords[:,1],np.arange(2,blue_ts_norm.shape[1]-2,1)) ) ) ]
# len(coords)
# 2- créer un tableau avec les patchs
train_X = []
for coord in coords :
    lst = []
    for band in [blue_ts_norm,green_ts_norm,red_ts_norm,nir_ts_norm]:
        lst.append( band[coord[0]-2:coord[0]+3,coord[1]-2:coord[1]+3] )
    train_X.append(np.stack(lst,axis=-1))#.reshape(20,20,-1))
train_X = np.stack(train_X,axis=0)
train_X.shape

In [ ]:
# Validation
# 1- précaution pour ne pas considérer les pixels dont on ne peut extraire des patchs car trop près du bord
idx_col = idx_valid[0]
idx_row = idx_valid[1]
coords = np.column_stack((idx_col,idx_row))
coords = coords[np.where( ( np.isin(coords[:,0],np.arange(2,blue_ts_norm.shape[0]-2,1)) ) & ( np.isin(coords[:,1],np.arange(2,blue_ts_norm.shape[1]-2,1)) ) ) ]
# len(coords)
# 2- créer un tableau avec les patchs
valid_X = []
for coord in coords :
    lst = []
    for band in [blue_ts_norm,green_ts_norm,red_ts_norm,nir_ts_norm]:
        lst.append( band[coord[0]-2:coord[0]+3,coord[1]-2:coord[1]+3] )
    valid_X.append(np.stack(lst,axis=-1))#.reshape(20,20,-1))
valid_X = np.stack(valid_X,axis=0)
valid_X.shape

In [ ]:
# Test
# 1- précaution pour ne pas considérer les pixels dont on ne peut extraire des patchs car trop près du bord
idx_col = idx_test[0]
idx_row = idx_test[1]
coords = np.column_stack((idx_col,idx_row))
coords = coords[np.where( ( np.isin(coords[:,0],np.arange(2,blue_ts_norm.shape[0]-2,1)) ) & ( np.isin(coords[:,1],np.arange(2,blue_ts_norm.shape[1]-2,1)) ) ) ]
# len(coords)
# 2- créer un tableau avec les patchs
test_X = []
for coord in coords :
    lst = []
    for band in [blue_ts_norm,green_ts_norm,red_ts_norm,nir_ts_norm]:
        lst.append( band[coord[0]-2:coord[0]+3,coord[1]-2:coord[1]+3] )
    test_X.append(np.stack(lst,axis=-1))#.reshape(20,20,-1))
test_X = np.stack(test_X,axis=0)
test_X.shape

In [ ]:
try:
    os.stat('Data/train_valid_test')
except:
    os.mkdir('Data/train_valid_test')

np.save('Data/train_valid_test/train_X.npy',train_X)
np.save('Data/train_valid_test/train_y.npy',train_y)

np.save('Data/train_valid_test/valid_X.npy',valid_X)
np.save('Data/train_valid_test/valid_y.npy',valid_y)
np.save('Data/train_valid_test/valid_id.npy',valid_id_array)

np.save('Data/train_valid_test/test_X.npy',test_X)
np.save('Data/train_valid_test/test_id.npy',test_id_array)